In [ ]:
import os
import pandas as pd
import csv
import numpy as np

In [ ]:
data_folder="C:\\Users\\Downloads\\diabetes-data.tar"
df_variable={}
df_list=[]
for file in os.listdir(data_folder):
    file_path=os.path.join(data_folder,file)
    df_variable[file]=f'df_{file}'
    header=['Misc','time','Predictor','Predictor_value']
    df_variable[file]=pd.read_csv(file_path,header=None)
    df_variable[file].columns=header
    df_variable[file].drop(columns=['Misc','time'],inplace=True)
    df_variable[file].sort_values(by='Predictor',ascending=True,inplace=True)
    df_variable[file]['Predictor_value']=df_variable[file]['Predictor_value'].iloc[df_variable[file].index]
    df_list.append(df_variable[file])
final=pd.concat(df_list,axis=0)
final_dia_data=final.set_index('Predictor').transpose()

In [ ]:
columns = [str(x) for x in list(set(final_dia_data.columns))]
column_data_var = []
clean_dict = {}
for i in range (len(columns)):
    column_data_var.append(f'{columns[i]}_data')
    column_data_var[i] = final_dia_data.filter(like=columns[i]).stack().values
    clean_dict[columns[i]] = column_data_var[i]
max_length = max(len(lst) for lst in clean_dict.values())
for key in clean_dict:
    clean_dict[key] = np.append(clean_dict[key], [0] * (max_length - len(clean_dict[key])))
clean_df=pd.DataFrame(clean_dict)
clean_df.drop(columns=['0','4','56','59','60','61','48','57','36','65','66','67','68','69','70','71','72','64'],inplace=True)

In [ ]:
cluster_df=clean_df.iloc[0:218,:]
cluster_df.rename(columns={'33':'Regular insulin dose','34':'NPH insulin dose','35':'UltraLente insulin dose','62':'Pre supper glucose measurement','63':'Post supper glucose measurement','58':'Pre breakfast glucose measurement'},inplace=True)

In [ ]:
list_val = cluster_df['Pre supper glucose measurement'].values.tolist()
rem_val_index=[]
for value in list_val:
    if isinstance(value, str) and not value.isdigit():
        rem_val_index.append(list_val.index(value))
        list_val.pop(list_val.index(value))
list_val=[int(val) for val in list_val]
mean_val=round(sum(list_val)/len(list_val))
for i in range(len(rem_val_index)):
    list_val.insert(rem_val_index[i],mean_val)
cluster_df['Pre supper glucose measurement']=list_val

In [ ]:
cluster_df